In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

import sys
sys.path.append('../')
from functions.plotting import *
from functions.filtering import *

from scipy.ndimage import gaussian_filter1d

In [2]:
#@title Figure settings
import ipywidgets as widgets

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [3]:
#@title Helper functions

def plot_weights(models, sharey=True):
  """Draw a stem plot of weights for each model in models dict."""
  n = len(models)
  f = plt.figure(figsize=(10, 2.5 * n))
  axs = f.subplots(n, sharex=True, sharey=sharey)
  axs = np.atleast_1d(axs)

  for ax, (title, model) in zip(axs, models.items()):

    ax.margins(x=.02)
    stem = ax.stem(model.coef_.squeeze(), use_line_collection=True)
    stem[0].set_marker(".")
    stem[0].set_color(".2")
    stem[1].set_linewidths(.5)
    stem[1].set_color(".2")
    stem[2].set_visible(False)
    ax.axhline(0, color="C3", lw=3)
    ax.set(ylabel="Weight", title=title)
  ax.set(xlabel="Neuron (a.k.a. feature)")
  f.tight_layout()


def plot_function(f, name, var, points=(-10, 10)):
    """Evaluate f() on linear space between points and plot.

    Args:
      f (callable): function that maps scalar -> scalar
      name (string): Function name for axis labels
      var (string): Variable name for axis labels.
      points (tuple): Args for np.linspace to create eval grid.
    """
    x = np.linspace(*points)
    ax = plt.figure().subplots()
    ax.plot(x, f(x))
    ax.set(
      xlabel=f'${var}$',
      ylabel=f'${name}({var})$'
    )


def plot_model_selection(C_values, accuracies):
  """Plot the accuracy curve over log-spaced C values."""
  ax = plt.figure().subplots()
  ax.set_xscale("log")
  ax.plot(C_values, accuracies, marker="o")
  best_C = C_values[np.argmax(accuracies)]
  ax.set(
      xticks=C_values,
      xlabel="$C$",
      ylabel="Cross-validated accuracy",
      title=f"Best C: {best_C:1g} ({np.max(accuracies):.2%})",
  )

def plot_non_zero_coefs(C_values, non_zero_l1, n_voxels):
  """Plot the accuracy curve over log-spaced C values."""
  ax = plt.figure().subplots()
  ax.set_xscale("log")
  ax.plot(C_values, non_zero_l1, marker="o")
  ax.set(
    xticks=C_values,
    xlabel="$C$",
    ylabel="Number of non-zero coefficients",
  )
  ax.axhline(n_voxels, color=".1", linestyle=":")
  ax.annotate("Total\n# Neurons", (C_values[0], n_voxels * .98), va="top")

In [4]:
def compute_accuracy(X, y, model):
  """Compute accuracy of classifier predictions.
  
  Args:
    X (2D array): Data matrix
    y (1D array): Label vector
    model (sklearn estimator): Classifier with trained weights.
  Returns:
    accuracy (float): Proportion of correct predictions.  
  """
  y_pred = model.predict(X)
  accuracy = (y == y_pred).mean()

  return accuracy

In [5]:
# load data from steinmetz dir
alldat = np.load('../steinmetz/steinmetz_part0.npz', allow_pickle=True)['dat']
alldat = np.hstack((alldat, np.load('../steinmetz/steinmetz_part1.npz', allow_pickle=True)['dat']))
alldat = np.hstack((alldat, np.load('../steinmetz/steinmetz_part2.npz', allow_pickle=True)['dat']))

In [107]:
cr_top10 = ["ZI", "APN", "MRN", "SCm", "PO", "LD", "SNr", "SSp", "MOp", "MOs"]
cr_others = ["SCs", "MG", "VPM", "VPL", "MD","CP", "PL", "ACA", "RSP", "VISam"]

In [20]:
filter_data_visp = filter_trials_full_contrast(alldat, "VISp") #recordings_with_region( alldat, "VISp")
# [
# mouse_name,
# mouse_spikes,
# mouse_regions,
# mouse_gocue,
# mouse_resptime,
# mouse_wheel,
# mouse_feedback,
# mouse_response,
# ]

In [21]:
#teste[0][0]
print(filter_data_visp.shape)
print(filter_data_visp[0][1].shape) # spikes
print(filter_data_visp[0][-1].shape) # response

(12, 8)
(178, 214, 250)
(214,)


In [24]:
# First define the model
log_reg = LogisticRegression(penalty="none")
cross_valid_k = 8 

region_neurons = []
mean_neurons_acc = []
neuron_choose = []

for animal_idx in range(filter_data_visp.shape[0]):
    for neuron_idx in range(filter_data_visp[animal_idx][1].shape[0]):
        X = filter_data_visp[animal_idx][1][neuron_idx] # spikes
        y = filter_data_visp[animal_idx][-1] # response
        print(X.shape, y.shape)
        accuracies = cross_val_score(LogisticRegression(penalty='none'), X, y, cv=cross_valid_k) 
        mean_neurons_acc.append(np.mean(accuracies))
    break
    
neuron_choose.append(np.argmax(mean_neurons_acc))
print(neuron_choose)

(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)
(214, 250) (214,)


KeyboardInterrupt: 

In [18]:
'''
# First define the model
log_reg = LogisticRegression(penalty="none")
cross_valid_k = 8 
# get neurons count for all regions
# all_rs = np.hstack([cr_top10, cr_others])
neuron_count = []

# region = all_rs[0]

#for region in all_rs:
region_neurons = []
mean_neurons_acc = []
neuron_choose = []

for dat in alldat:
    neurons = dat['brain_area'] == region
    neurons_spks = dat['spks'][neurons] # select one neuron
    ## Need to check if neurons_spks is null
    if(neurons_spks.shape[0] == 0):
        continue
    else:
        for i in range(neurons_spks.shape[0]):
            X = neurons_spks[i]
            y = dat['response']
            # print(X.shape, y.shape)
            accuracies = cross_val_score(LogisticRegression(penalty='none'), X, y, cv=cross_valid_k) 
            mean_neurons_acc.append(np.mean(accuracies))
    neuron_choose.append(np.argmax(mean_neurons_acc))
    print(neuron_choose)
'''

'\n# First define the model\nlog_reg = LogisticRegression(penalty="none")\ncross_valid_k = 8 \n# get neurons count for all regions\n# all_rs = np.hstack([cr_top10, cr_others])\nneuron_count = []\n\n# region = all_rs[0]\n\n#for region in all_rs:\nregion_neurons = []\nmean_neurons_acc = []\nneuron_choose = []\n\nfor dat in alldat:\n    neurons = dat[\'brain_area\'] == region\n    neurons_spks = dat[\'spks\'][neurons] # select one neuron\n    ## Need to check if neurons_spks is null\n    if(neurons_spks.shape[0] == 0):\n        continue\n    else:\n        for i in range(neurons_spks.shape[0]):\n            X = neurons_spks[i]\n            y = dat[\'response\']\n            # print(X.shape, y.shape)\n            accuracies = cross_val_score(LogisticRegression(penalty=\'none\'), X, y, cv=cross_valid_k) \n            mean_neurons_acc.append(np.mean(accuracies))\n    neuron_choose.append(np.argmax(mean_neurons_acc))\n    print(neuron_choose)\n'